# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


"The new Blackwell GPU from Nvidia features 5th Gen Tensor Cores with high-speed FP4, compute, and up to 4000 AI TOPS of performance, 4th Gen RT cores with up to 360 RT TFLOPs designed for Mega Geometry, a next-gen AI Management Processor for simultaneous AI models and graphics workloads, a new Blackwell SM with 125 TFLOPS of peak FP32 compute, and GDDR7 memory offering the world's fastest memory speeds of up to 30 Gbps. It also includes DLSS 4 and FP4 quantization for enhanced gaming performance and efficiency."

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [4]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

returning previous results due to exception reaching ddg.
https://weather.com/weather/today/l/USCA0987:1:US
https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8


In [5]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [6]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
# print(str(soup.body)[:50]) # limit long outputs

returning previous results due to exception reaching ddg.
Website: https://weather.com/weather/today/l/USCA0987:1:US




In [7]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/USCA0987:1:US


recents Weather Forecasts Radar & Maps News & Media Products Health & Wellness Account Lifestyle Privacy Specialty Forecasts San Francisco, CA Today's Forecast for San Francisco, CA Morning Afternoon Evening Overnight Weather Today in San Francisco, CA 7:20 am 5:22 pm Don't Miss Hourly Forecast Now 3 pm 4 pm 5 pm 6 pm Winter Central Trending Now Daily Forecast Today Thu 23 Fri 24 Sat 25 Sun 26 Radar We Love Our Critters Wellness In Winter Home, Garage & Garden How Does Yours Rank? Health News For You Happening Near San Francisco, CA Popular Nextdoor posts Rain Or Shine, It's Playtime Stay Safe Sick After The Rush? Weather in your inbox Your local forecast, plus daily trivia, stunning photos and our meteorologists’ top picks. All in one place, every weekday morning. By signing up, you're opting in to receive the Morning Brief email newsletter. To manage your data, visit Data Rights . Terms of Use | Privacy Policy Air Quality I

## Agentic Search

In [8]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1737584942, 'localtime': '2025-01-22 14:29'}, 'current': {'last_updated_epoch': 1737584100, 'last_updated': '2025-01-22 14:15', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 18, 'wind_dir': 'NNE', 'pressure_mb': 1027.0, 'pressure_in': 30.33, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 31, 'cloud': 25, 'feelslike_c': 16.1, 'feelslike_f': 61.0, 'windchill_c': 14.2, 'windchill_f': 57.6, 'heatindex_c': 13.9, 'heatindex_f': 57.0, 'dewpoint_c': 8.7, 'dewpoint_f': 47.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.7, 'gust_mph': 8.4, 'gust_kph': 13.6}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.775,
        "lon": -122.4183,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1737584942,
        "localtime": "2025-01-22 14:29"
    },
    "current": {
        "last_updated_epoch": 1737584100,
        "last_updated": "2025-01-22 14:15",
        "temp_c": 16.1,
        "temp_f": 61.0,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 5.6,
        "wind_kph": 9.0,
        "wind_degree": 18,
        "wind_dir": "NNE",
        "pressure_mb": 1027.0,
        "pressure_in": 30.33,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 31,
        "cloud": 25,
        "feelslike_c": 16.1,
        "feelslike_f": 61.0,
        "windchill_c": 14.2,
      

<img src="./google_sample.png" width="800" height="600">